# Data pre-processing project
The object of this project is to merge data from multiple folders into one DataFrame. In the process duplicates need to be dropped and missing latitude and longitude inserted from Google API. Finally the DataFrame should be saved to single csv file.

## Import

In [2]:
import os
import requests
import time

import pandas as pd
import numpy as np
import regex as re
import functools

## Read csv contents
Create one dataframe with all data

In [3]:
dataset_directory = "data"

dfs = []

# assume only subdirectories are in dataset_directory
for dir in os.listdir(dataset_directory):
    dir_path = os.path.join(dataset_directory, dir)
    addresses = pd.read_csv(os.path.join(dir_path, "ADDRESSES.csv"), sep=",")
    addresses_people = pd.read_csv(os.path.join(dir_path, "ADDRESSES_PEOPLE.csv"), sep=",")
    people = pd.read_csv(os.path.join(dir_path, "PEOPLE.csv"), sep=",")
    people_publications = pd.read_csv(os.path.join(dir_path, "PEOPLE_PUBLICATIONS.csv"), sep=",")
    publications = pd.read_csv(os.path.join(dir_path, "PUBLICATIONS.csv"), sep=",")

    df_dir = addresses_people\
         .merge(addresses, left_on="address_uuid", right_on="temp_id").drop(columns=["address_uuid", "temp_id"])\
         .merge(people, left_on="person_uuid", right_on="temp_id", suffixes=("_address", "_person")).drop(columns=["person_uuid"])\
         .merge(people_publications, left_on="temp_id", right_on="person_uuid").drop(columns=["person_uuid", "temp_id"])\
         .merge(publications, left_on="publication_uuid", right_on="temp_id").drop(columns=["publication_uuid", "temp_id"])

    dfs.append(df_dir)

df = pd.concat(dfs)
df.head(2)

,address,countries_scope_address,lat,lon,name,phone,url_address,lastname,firstname,countries_scope_person,email,town,role,url_person,title,pubmed_id,journal,year,url,abstract
0,"norwegian school of sport sciencesoslo, norway",NO,NaN,NaN,norwegian research centre for training and per...,NaN,NaN,Losnegard,Thomas,NO,thomas.losnegard@nih.no,oslo,NaN,NaN,Anaerobic capacity as a determinant of perform...,21952633,Medicine and science in sports and exercise. V...,2012.0,http://www.ncbi.nlm.nih.gov/pubmed/21952633/,As cross-country sprint competitions rely on m...
1,"0316, 1047, oslo, norway",NO,NaN,NaN,"department of geosciences, university of oslo,...",NaN,NaN,Westermann,Sebastian,NO,NaN,oslo,NaN,NaN,Contrasting temperature trends across the ice-...,29371633,Scientific reports. Volume: 8 Issue: 1 01 25 2018,2018.0,http://www.ncbi.nlm.nih.gov/pubmed/29371633/,Temperature changes in the Arctic have notable...


## Reset index

In [4]:
df.reset_index(drop=True, inplace=True)

## Merge same addresses into one

### Drop same rows

In [5]:
df["address"] = df["address"].apply(lambda a: re.sub(r'[^\p{L}\p{N}\s,]', "", a))
df["address"] = df["address"].apply(lambda a: re.sub(r'( |,)[\p{L}]( |,)', " ", a))
df["address"] = df["address"].apply(lambda a: a.strip().strip(","))
initial_rows = df.shape[0]
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

print(f"Dropped {initial_rows - df.shape[0]} rows")

Dropped 20140 rows


### Group exactly the same addresses

In [6]:
addrs = df.groupby('address').apply(lambda x: x.index.tolist()).reset_index(name="indices")
addrs

,address,indices
0,"1050, oslo, norway",[217853]
1,"5305, 5021 bergen, norway",[111584]
2,"0, 0318, oslo, norway",[42103]
3,"0015, 1550, oslo, norway",[227608]
4,"0028, 9569, oslo, norway",[30974]
...,...,...
25429,"øya helsehustrondheim, norway",[134475]
25430,"øysteinsgate 3, 5007, bergen, norway","[104614, 185845]"
25431,"øysteinsgate 3, 5199, 5007 bergen, norway",[15966]
25432,"øysteinsgate 3bergen, norway","[49168, 49169]"


### Find the addresses that are suppose to be the same

In [7]:
from Levenshtein import distance

### Create matrix with distance betweend each address

In [8]:
lev_distances = np.vectorize(distance)(addrs["address"].to_numpy()[:, np.newaxis], addrs["address"].to_numpy()[np.newaxis, :])

In [9]:
mat = pd.DataFrame(np.triu(lev_distances, k=1), index=range(lev_distances.shape[0]), columns=range(lev_distances.shape[1]))
mat

,0,1,2,3,4,5,6,7,8,9,...,25424,25425,25426,25427,25428,25429,25430,25431,25432,25433
0,0,13,6,6,9,8,8,9,9,9,...,19,21,21,20,18,18,23,28,19,34
1,0,0,13,14,15,13,12,14,14,15,...,12,27,27,26,25,20,17,20,13,26
2,0,0,0,7,7,7,7,6,6,7,...,20,21,21,20,19,18,24,29,19,34
3,0,0,0,0,5,6,6,6,6,6,...,19,21,21,20,17,20,23,27,19,34
4,0,0,0,0,0,6,6,6,6,6,...,20,21,21,21,19,20,24,28,19,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25429,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,23,28,17,33
25430,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,7,8,14
25431,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,13,14
25432,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,19


Filter these addresses that have distance in specified range

In [10]:
upper_thresh = 3
below_thresh = mat.apply(lambda r: r[r.between(0, upper_thresh, inclusive='neither')].index.to_numpy(), axis=1)
below_thresh = below_thresh[below_thresh.apply(lambda x: len(x)> 0)]
below_thresh

2        [11793, 11795, 11796, 11797, 11798, 11799, 118...
5                                                      [6]
8                        [9, 11, 12, 13, 14, 11985, 12774]
9                           [11, 12, 13, 14, 11986, 11987]
10                                                    [11]
                               ...                        
25378                                              [25379]
25388                                              [25389]
25395                                              [25396]
25425                                              [25426]
25426                                              [25427]
Length: 7113, dtype: object

## Update Dataframe

In [11]:
already_changed = set()

for idx, ridxs in below_thresh.items():
    if idx not in already_changed:
        for ridx in ridxs:
            df.loc[addrs.iloc[ridx]["indices"], "address"] = addrs.iloc[idx]["address"]
        already_changed = already_changed.union(set(ridxs))

## Fill missing lat/lon

In [12]:
towns = set(line.strip().lower() for line in open('towns.txt'))

In [13]:
@functools.cache
def get_town(string):
    splits = re.sub(r'[^\p{L}\s]', "", string).split()
    for s in splits[::-1]:
        s = s.lower()
        if s in towns:
            return s
    return ""


In [14]:
old_towns = df["town"]
old_towns.head(3)

0    oslo
1    oslo
2      as
Name: town, dtype: object

In [15]:
df["town"] = df["town"].fillna("").apply(lambda t: get_town(t))
df.loc[df["town"] == "", "town"] = df.loc[df["town"] == "", "address"].apply(lambda a: get_town(a))
df.loc[df["town"] == "", "town"] = old_towns[df["town"] == ""]

In [16]:
df["town"]

0           oslo
1           oslo
2             as
3           oslo
4         tromsø
           ...  
237429      oslo
237430      oslo
237431    tromsø
237432    bergen
237433      oslo
Name: town, Length: 237434, dtype: object

In [35]:
@functools.cache
def get_lat_lon(town):
    query_params = {
        "text": town,
        "apiKey": #YOUR_KEY,
    }

    r = requests.get(
        "https://api.geoapify.com/v1/geocode/search",
        params=query_params,
    )
    
    # If free version is used
    # time.sleep(1/4)
    
    if "features" in r.json() and len(r.json()["features"]) > 0:
        return r.json()["features"][0]["properties"]["lat"], r.json()["features"][0]["properties"]["lon"]
    return np.nan, np.nan

In [42]:
# df[["lat", "lon"]] = df.apply(lambda r: pd.Series(get_lat_lon(r['town'])), axis=1)

In [46]:
df.tail(7)

,address,countries_scope_address,lat,lon,name,phone,url_address,lastname,firstname,countries_scope_person,email,town,role,url_person,title,pubmed_id,journal,year,url,abstract
237427,"modum bad research institutevikersund, norway",NO,63.430447,10.395212,NaN,NaN,NaN,Bergseng,H,NO,hakon.bergseng@ntnu.no,trondheim,NaN,NaN,Molecular and phenotypic characterization of i...,19456824,Clinical microbiology and infection : the offi...,2009.0,http://www.ncbi.nlm.nih.gov/pubmed/19456824/,Multilocus sequence typing of an almost comple...
237428,norwegian university for science and technolog...,NO,63.430447,10.395212,"department of laboratory medicine, children's ...",NaN,NaN,Bergseng,H,NO,hakon.bergseng@ntnu.no,trondheim,NaN,NaN,Molecular and phenotypic characterization of i...,19456824,Clinical microbiology and infection : the offi...,2009.0,http://www.ncbi.nlm.nih.gov/pubmed/19456824/,Multilocus sequence typing of an almost comple...
237429,"university of oslooslo 3, norway",NO,59.913330,10.738970,department of psychology,NaN,NaN,Lima,K,NO,kari.lima@medisin.uio.no,oslo,NaN,NaN,Low thymic output in the 22q11.2 deletion synd...,20491792,Clinical and experimental immunology. Volume: ...,2010.0,http://www.ncbi.nlm.nih.gov/pubmed/20491792/,Thymic hypoplasia is a frequent feature of the...
237430,"university of oslooslo 3, norway",NO,59.913330,10.738970,"section of endocrinology, faculty division ake...",NaN,NaN,Lima,K,NO,kari.lima@medisin.uio.no,oslo,NaN,NaN,Low thymic output in the 22q11.2 deletion synd...,20491792,Clinical and experimental immunology. Volume: ...,2010.0,http://www.ncbi.nlm.nih.gov/pubmed/20491792/,Thymic hypoplasia is a frequent feature of the...
237431,"university of tromso, norway",NO,69.651635,18.955859,NaN,NaN,NaN,Silvera,D H,NO,NaN,tromsø,NaN,NaN,Analyzing the relation between self-esteem and...,10728157,Eating and weight disorders : EWD. Volume: 3 I...,1998.0,http://www.ncbi.nlm.nih.gov/pubmed/10728157/,Research in the past has demonstrated an assoc...
237432,department of somatic psychologyuniversity of ...,NO,60.394306,5.325919,NaN,NaN,NaN,Svebak,S,NO,NaN,bergen,NaN,NaN,The personality of the cardiac responder: inte...,3447633,Biological psychology. Volume: 25 Issue: 1 Aug...,1987.0,http://www.ncbi.nlm.nih.gov/pubmed/3447633/,Forty healthy university students were recruit...
237433,"ullveien 14, 0791, oslo, norway",NO,59.913330,10.738970,"voksentoppen bkl, rikshospitalet university ho...",NaN,NaN,Carlsen,Kai-Håkon,NO,k.h.carlsen@medisin.uio.no,oslo,NaN,NaN,Pharmaceutical treatment of asthma in children.,16248823,Current drug targets. Inflammation and allergy...,2005.0,http://www.ncbi.nlm.nih.gov/pubmed/16248823/,The present review article gives an overview o...


## Save Dataframe to csv

In [44]:
df.to_csv("data.csv")

In [45]:
df

,address,countries_scope_address,lat,lon,name,phone,url_address,lastname,firstname,countries_scope_person,email,town,role,url_person,title,pubmed_id,journal,year,url,abstract
0,"norwegian school of sport sciencesolso, norway",NO,59.913330,10.738970,norwegian research centre for training and per...,NaN,NaN,Losnegard,Thomas,NO,thomas.losnegard@nih.no,oslo,NaN,NaN,Anaerobic capacity as a determinant of perform...,21952633,Medicine and science in sports and exercise. V...,2012.0,http://www.ncbi.nlm.nih.gov/pubmed/21952633/,As cross-country sprint competitions rely on m...
1,"0316, 1046, oslo, norway",NO,59.913330,10.738970,"department of geosciences, university of oslo,...",NaN,NaN,Westermann,Sebastian,NO,NaN,oslo,NaN,NaN,Contrasting temperature trends across the ice-...,29371633,Scientific reports. Volume: 8 Issue: 1 01 25 2018,2018.0,http://www.ncbi.nlm.nih.gov/pubmed/29371633/,Temperature changes in the Arctic have notable...
2,"agricultural university of norwayaas, norway",NO,NaN,NaN,department of plant and environmental sciences,NaN,NaN,Bjørnstad,Asmund,NO,NaN,as,NaN,NaN,Analysis of genetic marker-phenotype relations...,15660976,Hereditas. Volume: 141 Issue: 2 2004,2004.0,http://www.ncbi.nlm.nih.gov/pubmed/15660976/,The utility of a relatively new multivariate m...
3,"red cross clinicoslo, norway",NO,59.913330,10.738970,department of internal medicine,NaN,NaN,Andersen,P,NO,NaN,oslo,NaN,NaN,Increased fibrinolytic potential after diet in...,3389205,Acta medica Scandinavica. Volume: 223 Issue: 6...,1988.0,http://www.ncbi.nlm.nih.gov/pubmed/3389205/,Twenty healthy individuals (15 men and 5 women...
4,"northern research institute norut, 9292, troms...",NO,69.651635,18.955859,NaN,NaN,NaN,Fernandez-Luque,Luis,NO,luis.luque@norut.no,tromsø,NaN,NaN,An analysis of personal medical information di...,19745316,Studies in health technology and informatics. ...,2009.0,http://www.ncbi.nlm.nih.gov/pubmed/19745316/,The Internet has become one of the main source...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237429,"university of oslooslo 3, norway",NO,59.913330,10.738970,department of psychology,NaN,NaN,Lima,K,NO,kari.lima@medisin.uio.no,oslo,NaN,NaN,Low thymic output in the 22q11.2 deletion synd...,20491792,Clinical and experimental immunology. Volume: ...,2010.0,http://www.ncbi.nlm.nih.gov/pubmed/20491792/,Thymic hypoplasia is a frequent feature of the...
237430,"university of oslooslo 3, norway",NO,59.913330,10.738970,"section of endocrinology, faculty division ake...",NaN,NaN,Lima,K,NO,kari.lima@medisin.uio.no,oslo,NaN,NaN,Low thymic output in the 22q11.2 deletion synd...,20491792,Clinical and experimental immunology. Volume: ...,2010.0,http://www.ncbi.nlm.nih.gov/pubmed/20491792/,Thymic hypoplasia is a frequent feature of the...
237431,"university of tromso, norway",NO,69.651635,18.955859,NaN,NaN,NaN,Silvera,D H,NO,NaN,tromsø,NaN,NaN,Analyzing the relation between self-esteem and...,10728157,Eating and weight disorders : EWD. Volume: 3 I...,1998.0,http://www.ncbi.nlm.nih.gov/pubmed/10728157/,Research in the past has demonstrated an assoc...
237432,department of somatic psychologyuniversity of ...,NO,60.394306,5.325919,NaN,NaN,NaN,Svebak,S,NO,NaN,bergen,NaN,NaN,The personality of the cardiac responder: inte...,3447633,Biological psychology. Volume: 25 Issue: 1 Aug...,1987.0,http://www.ncbi.nlm.nih.gov/pubmed/3447633/,Forty healthy university students were recruit...
